In [13]:
if (!require("data.table")) install.packages("data.table")
library(data.table)

In [4]:
all_species_tissue_data_1 <- readRDS("all_species_tissue_data_1.RDS")

In [17]:
process_alternative_splicing_jupyter <- function(expression_file, as_data) {
  # Jupyter专用输出函数 - 确保输出显示
  jupyter_print <- function(msg) {
    cat(msg, "\n")
    flush.console()  # 强制刷新输出缓冲区
  }
  
  start_time <- Sys.time()
  jupyter_print("开始处理可变剪切数据分析...")
  
  # 读取基因表达数据并转换为data.table
  jupyter_print(paste("读取基因表达文件:", expression_file, "..."))
  gene_expr <- fread(expression_file)
  jupyter_print(paste("成功加载基因表达数据，共", nrow(gene_expr), "行。"))
  
  # 预先添加结果列
  gene_expr[, `:=`(AS_count = 0L, dPSI_count = 0L)]
  
  # 清除基因ID中可能的前缀（如"+"）- 向量化处理
  jupyter_print("预处理基因表达数据...")
  gene_expr[, clean_gene_id := gsub("^\\+", "", GeneID)]
  
  # 预先构建所有需要的列名组合 - 向量化处理
  jupyter_print("构建阶段列名...")
  gene_expr[, prev_column := paste0(prev_main_stage, "-", prev_sub_stage, "-", tissue)]
  gene_expr[, current_column := paste0(main_stage, "-", sub_stage, "-", tissue)]
  
  # 转换as_data为data.table并创建索引
  jupyter_print("预处理可变剪切数据并创建索引...")
  as_data_dt <- list()
  tissue_count <- length(names(as_data))
  
  # 为进度条创建独立单元格输出 (Jupyter特有)
  jupyter_print("") # 创建空行用于进度条
  
  # 预处理特殊索引 - 优化匹配速度
  for (i in seq_along(as_data)) {
    tissue_name <- names(as_data)[i]
    progress_msg <- sprintf("\r[%d/%d] 处理组织 '%s'...", i, tissue_count, tissue_name)
    cat(progress_msg)
    flush.console()
    
    if (is.data.frame(as_data[[tissue_name]])) {
      dt <- as.data.table(as_data[[tissue_name]])
      
      # 提取基因ID部分并创建索引列表结构 - 高效查询
      dt[, gene_id := gsub("^[^_]+_([^_]+)_.*$", "\\1", Event_ID)]
      
      # 创建高效查询结构 - 对每个基因预先存储事件索引
      gene_indices <- split(seq_len(nrow(dt)), dt$gene_id)
      
      # 存储数据表和索引结构
      as_data_dt[[tissue_name]] <- list(
        data = dt,
        indices = gene_indices
      )
    }
  }
  cat("\n") # 完成进度条
  jupyter_print(paste("预处理完成，共处理", length(as_data_dt), "个组织。"))
  
  # 分组处理 - 按组织批量处理
  jupyter_print("\n开始批量处理数据...")
  unique_tissues <- unique(gene_expr$tissue)
  
  # 创建总进度跟踪
  total_tissues <- length(unique_tissues)
  overall_progress <- 0
  
  for (t_idx in seq_along(unique_tissues)) {
    curr_tissue <- unique_tissues[t_idx]
    tissue_start <- Sys.time()
    jupyter_print(sprintf("\n[组织 %d/%d] 处理组织 '%s'...", 
                         t_idx, total_tissues, curr_tissue))
    
    # 如果当前组织不存在于AS数据中，跳过
    if (!curr_tissue %in% names(as_data_dt)) {
      jupyter_print(sprintf("  - 跳过组织 '%s': 无相关AS数据", curr_tissue))
      next
    }
    
    # 筛选当前组织的表达数据
    tissue_expr <- gene_expr[tissue == curr_tissue]
    jupyter_print(sprintf("  - 找到 %d 行相关表达数据", nrow(tissue_expr)))
    
    tissue_data <- as_data_dt[[curr_tissue]]$data
    gene_indices <- as_data_dt[[curr_tissue]]$indices
    
    # 使用apply迭代每个基因 - 比for循环更高效
    jupyter_print("  - 开始批量计算AS计数和dPSI值...")
    batch_size <- 500  # 设置合适的批次大小
    num_batches <- ceiling(nrow(tissue_expr) / batch_size)
    
    # Jupyter进度条 - 创建空行
    cat("\r  - 批次进度: 0%")
    flush.console()
    
    for (batch_idx in 1:num_batches) {
      # 更新进度条 - Jupyter友好格式
      progress_pct <- round(batch_idx / num_batches * 100)
      cat(sprintf("\r  - 批次进度: %d%%", progress_pct))
      flush.console()
      
      batch_start <- (batch_idx - 1) * batch_size + 1
      batch_end <- min(batch_idx * batch_size, nrow(tissue_expr))
      
      # 处理当前批次
      for (i in batch_start:batch_end) {
        gene_id <- tissue_expr$clean_gene_id[i]
        prev_col <- tissue_expr$prev_column[i]
        curr_col <- tissue_expr$current_column[i]
        
        # 检查列是否存在
        if (!(prev_col %in% names(tissue_data)) || !(curr_col %in% names(tissue_data))) {
          next
        }
        
        # 查找相关事件
        if (gene_id %in% names(gene_indices)) {
          event_idx <- gene_indices[[gene_id]]
          
          # 记录AS事件总数
          gene_expr[tissue == curr_tissue & clean_gene_id == gene_id & 
                    prev_column == prev_col & current_column == curr_col, 
                   AS_count := length(event_idx)]
          
          # 快速计算dPSI - 无循环
          prev_psi <- tissue_data[[prev_col]][event_idx]
          current_psi <- tissue_data[[curr_col]][event_idx]
          
          # 同时存在有效PSI值的行
          valid_rows <- !is.na(prev_psi) & !is.na(current_psi)
          if (sum(valid_rows) > 0) {
            dpsi_values <- abs(current_psi[valid_rows] - prev_psi[valid_rows])
            dpsi_count <- sum(dpsi_values > 0.2, na.rm = TRUE)
            
            # 更新结果
            gene_expr[tissue == curr_tissue & clean_gene_id == gene_id & 
                      prev_column == prev_col & current_column == curr_col, 
                     dPSI_count := dpsi_count]
          }
        }
      }
    }
    cat("\n") # 完成批次进度条
    
    tissue_time <- difftime(Sys.time(), tissue_start, units="mins")
    jupyter_print(sprintf("  - 组织 '%s' 处理完成，用时: %.2f分钟", 
                        curr_tissue, as.numeric(tissue_time)))
    
    # 更新总进度
    overall_progress <- t_idx / total_tissues * 100
    jupyter_print(sprintf("总进度: %.1f%% (%d/%d 组织)", 
                        overall_progress, t_idx, total_tissues))
  }
  
  # 移除临时列
  gene_expr[, c("clean_gene_id", "prev_column", "current_column") := NULL]
  
  # 报告总结
  end_time <- Sys.time()
  total_time <- round(as.numeric(difftime(end_time, start_time, units="mins")), 2)
  jupyter_print(sprintf("\n处理完成! 总用时: %.2f分钟", total_time))
  jupyter_print(sprintf("总基因数: %d, 有AS事件的基因数: %d, 有显著dPSI(>0.2)的基因数: %d",
                      nrow(gene_expr), 
                      sum(gene_expr$AS_count > 0), 
                      sum(gene_expr$dPSI_count > 0)))
  
  return(gene_expr)
}

In [19]:
result <- process_alternative_splicing_jupyter("gene_expression_fold_changes.csv", all_species_tissue_data_1[["Bombyx_mori"]])

开始处理可变剪切数据分析... 
读取基因表达文件: gene_expression_fold_changes.csv ... 
成功加载基因表达数据，共 1494574 行。 
预处理基因表达数据... 
构建阶段列名... 
预处理可变剪切数据并创建索引... 
 
[15/15] 处理组织 'Wing'...dy'...issue'...
预处理完成，共处理 15 个组织。 

开始批量处理数据... 

[组织 1/15] 处理组织 'Head'... 
  - 找到 129098 行相关表达数据 
  - 开始批量计算AS计数和dPSI值... 
  - 批次进度: 100%
  - 组织 'Head' 处理完成，用时: 3.41分钟 
总进度: 6.7% (1/15 组织) 

[组织 2/15] 处理组织 'Wing'... 
  - 找到 12575 行相关表达数据 
  - 开始批量计算AS计数和dPSI值... 
  - 批次进度: 100%
  - 组织 'Wing' 处理完成，用时: 0.00分钟 
总进度: 13.3% (2/15 组织) 

[组织 3/15] 处理组织 'Developmental_tissue'... 
  - 找到 107328 行相关表达数据 
  - 开始批量计算AS计数和dPSI值... 
  - 批次进度: 100%
  - 组织 'Developmental_tissue' 处理完成，用时: 2.66分钟 
总进度: 20.0% (3/15 组织) 

[组织 4/15] 处理组织 'Fat_body'... 
  - 找到 159228 行相关表达数据 
  - 开始批量计算AS计数和dPSI值... 
  - 批次进度: 100%
  - 组织 'Fat_body' 处理完成，用时: 4.92分钟 
总进度: 26.7% (4/15 组织) 

[组织 5/15] 处理组织 'Ovary'... 
  - 找到 60090 行相关表达数据 
  - 开始批量计算AS计数和dPSI值... 
  - 批次进度: 100%
  - 组织 'Ovary' 处理完成，用时: 1.15分钟 
总进度: 33.3% (5/15 组织) 

[组织 6/15] 处理组织 'Testis'... 
  - 找到 174

In [20]:
print(result)

         tissue GeneID prev_main_stage    prev_sub_stage main_stage
      1:   Head +nsd-2           Larva                        Larva
      2:   Head +nsd-2           Larva 0_day_of_5_instar      Larva
      3:   Head +nsd-2           Larva 2_day_of_4_instar      Larva
      4:   Head +nsd-2           Larva          2_instar      Larva
      5:   Head +nsd-2           Larva 3_day_of_4_instar      Larva
     ---                                                           
1494570:   Pupa    zen            Pupa      2_day_of_pre       Pupa
1494571:   Pupa    zen            Pupa            4_days       Pupa
1494572:   Pupa    zen            Pupa          7.8_days       Pupa
1494573:   Pupa    zen            Pupa             newly       Pupa
1494574:   Pupa    zen            Pupa               pre       Pupa
                 sub_stage fold_change expression AS_count dPSI_count
      1: 0_day_of_5_instar   3.4078400  -0.191400        0          0
      2: 2_day_of_4_instar  -3.2740667  -3.4

In [21]:
fwrite(result, "·")

In [23]:
# 加载必要的包
library(dplyr)

# 读取CSV文件，假设文件名为 "result.csv"
#df <- read.csv("result.csv", stringsAsFactors = FALSE)

# 按指定分组并统计每组的基因数和 dPSI_count 总和
results <- result %>%
  group_by(tissue, prev_main_stage, prev_sub_stage, main_stage, sub_stage) %>%
  summarise(
    gene_count = n_distinct(GeneID),      # 统计每组不同基因的数量
    total_dPSI_count = sum(dPSI_count)      # 统计每组 dPSI_count 的总和
  )

# 显示结果
print(results)

`summarise()` has grouped output by 'tissue', 'prev_main_stage', 'prev_sub_stage', 'main_stage'. You can override using the `.groups` argument.


# A tibble: 144 × 7
# Groups:   tissue, prev_main_stage, prev_sub_stage, main_stage [144]
   tissue         prev_main_stage prev_sub_stage main_stage sub_stage gene_count
   <chr>          <chr>           <chr>          <chr>      <chr>          <int>
 1 Developmental… Larva           ""             Larva      0_day_of…      12356
 2 Developmental… Larva           "0_day_of_5_i… Larva      36h_wand…      11823
 3 Developmental… Larva           "36h_wanderin… Larva      3_day_of…      10234
 4 Developmental… Larva           "3_day_of_4_i… Larva      3_day_of…       8103
 5 Developmental… Larva           "3_day_of_5_i… Larva      4_instar        9843
 6 Developmental… Larva           "4_instar"     Larva      52h_wand…      10372
 7 Developmental… Larva           "52h_wanderin… Larva      5_instar        9686
 8 Developmental… Larva           "5_instar"     Larva      day_6_of…      11120
 9 Developmental… Larva           "day_6_of_5_i… Larva      wanderin…      12153
10 Developmental… L

In [24]:
write.csv(results, "summary_result.csv", row.names = FALSE)
